In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
import datetime as dt
%matplotlib inline

from sklearn import metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV

In [ ]:
cust_data = pd.read_excel("C:\Users\Sushanta Nayak\OneDrive\Desktop\Case Studys\Machine Learning\14. Capstone Case Study - Finding-Marketing-Insights\CustomersData.xlsx")

In [ ]:
discount_coupon = pd.read_csv(r"C:\Users\Sushanta Nayak\OneDrive\Desktop\Case Studys\Machine Learning\14. Capstone Case Study - Finding-Marketing-Insights\Discount_Coupon.csv")

In [ ]:
marketing_spend = pd.read_csv(r"C:\Users\Sushanta Nayak\OneDrive\Desktop\Case Studys\Machine Learning\14. Capstone Case Study - Finding-Marketing-Insights\Marketing_Spend.csv")

In [ ]:
online_sales = pd.read_csv(r"C:\Users\Sushanta Nayak\OneDrive\Desktop\Case Studys\Machine Learning\14. Capstone Case Study - Finding-Marketing-Insights\Online_Sales.csv")

In [ ]:
tax_amt = pd.read_excel("C:\Users\Sushanta Nayak\OneDrive\Desktop\Case Studys\Machine Learning\14. Capstone Case Study - Finding-Marketing-Insights\Tax_amount.xlsx")

In [ ]:
cust_data.head(2)

In [ ]:
cust_data.info()

In [ ]:
discount_coupon.head(2)

In [ ]:
# removing space from columns name
discount_coupon.columns = discount_coupon.columns.str.replace(' ', '')

In [ ]:
discount_coupon.info()

In [ ]:
marketing_spend.head(2)

In [ ]:
marketing_spend.info()

In [ ]:
online_sales.head(2)

In [ ]:
# Changing Transaction_Date to datetime format

online_sales['Transaction_Date'] = pd.to_datetime(online_sales.Transaction_Date , format= '%Y%m%d' )
online_sales['Month'] = online_sales['Transaction_Date'].apply(lambda x : pd.Timestamp.strftime(x, format = '%b'))
online_sales.head(2)

In [ ]:
online_sales.info()

In [ ]:
tax_amt.head(2)

In [ ]:
tax_amt.info()

In [ ]:
# merging the data

online_sales1 = pd.merge(online_sales , tax_amt , how = 'left',  left_on= 'Product_Category', right_on='Product_Category')
online_sales2 = pd.merge(online_sales1 , cust_data, how = 'left',  left_on= 'CustomerID', right_on='CustomerID')
final_data = pd.merge(online_sales2 , discount_coupon, how = 'left', on= ['Month','Product_Category'])

In [ ]:
final_data.head()

## 1. Calculating Invoice amount


In [ ]:
final_data['Invoice_amt'] = ((final_data.Quantity*final_data.Avg_Price)*((1-final_data.Discount_pct/100))*(1+final_data.GST))+final_data.Delivery_Charges

In [ ]:
final_data['Sales_amt'] = ((final_data.Quantity*final_data.Avg_Price)*((1-final_data.Discount_pct/100)))

In [ ]:
final_data.head(2)

## 2. Exploratory Analysis


#### • Understanding how many new customers acquired every month


In [ ]:
final_data['Year_month'] = final_data.Transaction_Date.apply(lambda x: pd.Timestamp.strftime(x , format = "%Y-%m"))

In [ ]:
cust_month = final_data.loc[:, ['CustomerID', 'Year_month']] 

In [ ]:
cust_month.drop_duplicates(subset='CustomerID' , inplace= True)

In [ ]:
new_cust_monthly = cust_month.groupby('Year_month')[['CustomerID']].count()
new_cust_monthly.head(2)

In [ ]:
new_cust_monthly.plot( kind = 'bar', figsize = (18,8))
plt.ylabel('count of customers')
plt.title('New customers acquired every month')
plt.show()

In [ ]:
cust_monthly_cnt = final_data.groupby('Year_month')[['CustomerID']].count().reset_index()
cust_monthly_cnt.head(2)

In [ ]:
retented_cust_monthly = pd.merge(new_cust_monthly , cust_monthly_cnt , how= 'left' , on ="Year_month")
retented_cust_monthly.head(2)

In [ ]:
retented_cust_monthly['retented_cust'] = retented_cust_monthly.CustomerID_y - retented_cust_monthly.CustomerID_x
retented_cust_monthly.head(2)

In [ ]:
retented_cust_monthly.plot(kind= 'bar', x= 'Year_month' , y = 'retented_cust' , figsize = (16,6))
plt.ylabel('count of customers')
plt.title('Retention of customers on month on month basis')
plt.show()

In [ ]:
rev_mouth = final_data.groupby('Year_month')[['Sales_amt']].sum()
rev_mouth.head(2)

In [ ]:
rev_mouth.plot( kind = 'bar', figsize =(16,6))
plt.ylabel('Total Amount')
plt.title('Total Revenue on month on month basis')
plt.show()

### • How the revenue from new customer on monthly bases


In [ ]:
cust_month_invoice = final_data.loc[:, ['CustomerID', 'Year_month', 'Sales_amt']] 

In [ ]:
cust_month_invoice.drop_duplicates(subset='CustomerID' , inplace= True)

In [ ]:
new_cust_rev_monthly = cust_month_invoice.groupby('Year_month')[['Sales_amt']].sum()
new_cust_rev_monthly.head(2)

In [ ]:
new_cust_rev_monthly.plot( kind = 'bar', figsize = (18,5))
plt.ylabel('Revenue')
plt.title('New customers revenue every month')
plt.show()

### • How the discount playing the role in revenue?


In [ ]:
final_data.head(2)

In [ ]:
discount_rev = final_data.groupby('Discount_pct')[['Sales_amt']].sum()
discount_rev.head(2)

In [ ]:
discount_rev.plot( kind = 'bar', figsize = (10,5))
plt.ylabel('Revenue')
plt.show()

### • Understand the trends/seasonality of sales, quantity by category, month, week, day,etc…


In [ ]:
# Seasonality of sales, quantity by month.
cat_qty_sales = final_data.groupby('Year_month').agg({'Quantity':'sum',
                                                       'Sales_amt': 'sum'}).reset_index().rename(columns={'Sales_amt':'sales'})
round(cat_qty_sales,2)

In [ ]:
marketing_spend.head(2)

In [ ]:
marketing_spend.info()

In [ ]:
marketing_spend['Date'] = pd.to_datetime(marketing_spend.Date , format= '%m/%d/%Y' )

In [ ]:
marketing_spend['Year_month'] = marketing_spend.Date.apply(lambda x: pd.Timestamp.strftime(x , format = "%Y-%m"))

In [ ]:
marketing_spend['Total_spend'] = marketing_spend.Offline_Spend + marketing_spend.Online_Spend

In [ ]:
marketing_spend.head(2)

In [ ]:
montly_spend = marketing_spend.groupby('Year_month')['Total_spend'].sum().reset_index()
montly_spend.head(2)

In [ ]:
montly_rev = final_data.groupby('Year_month')['Sales_amt'].sum().reset_index()
montly_rev.head(2)

In [ ]:
pd.merge(montly_spend, montly_rev, how = 'left', on='Year_month' )

In [ ]:
plt.figure(figsize= (15,8))
plt.plot(montly_rev.Year_month, montly_rev.Sales_amt,   label = 'Revenue', marker='o', linewidth=3)
plt.plot(montly_spend.Year_month, montly_spend.Total_spend,   label = 'Spend',  marker='o', linewidth=3)

plt.xlabel('Month Number')
plt.legend(loc='upper left')
plt.show()

### • Which product was purchased most based on the quantity?


In [ ]:
prod_qty = final_data.groupby('Product_Category')['Quantity'].sum().reset_index()

In [ ]:
prod_qty.sort_values(by='Quantity', ascending=False).head(1)


## 3.Performing Customer Segmentation


In [ ]:
final_data.head(2)

In [ ]:
# Setting a date to extract recency 

date = dt.datetime(2020, 2, 20)

In [ ]:
cust_mf = final_data.groupby('CustomerID').agg({'Transaction_ID':'count',
                                                'Sales_amt':'sum'}).reset_index().rename(columns={'Transaction_ID': 'frequency', 'Sales_amt': 'monetary'})
cust_mf

In [ ]:
cust_recency = final_data.loc[:,['CustomerID','Transaction_Date']]

In [ ]:
cust_recency = cust_recency.drop_duplicates(subset='CustomerID', keep='last')

In [ ]:
cust_recency['recency'] = (date - cust_recency.Transaction_Date).dt.days

In [ ]:
seg_df = pd.merge(cust_recency, cust_mf, how= 'right', on= 'CustomerID')

In [ ]:
seg_df.drop(columns='Transaction_Date', inplace=True)
seg_df.head()

## Value based segmentation


In [ ]:
# creating Deciles

seg_df['decile'] = pd.qcut(seg_df['monetary'], 10, labels=False)

In [ ]:
seg_df.head()

In [ ]:
seg_df.decile.value_counts()

In [ ]:
# Key performace variable selection
col_kpi=['monetary','recency','frequency']

In [ ]:
#Profiling
VB_Profiling=seg_df.groupby(['decile']).apply(lambda x: x[col_kpi].mean())

In [ ]:
VB_Profiling.T

In [ ]:
profiling_output = VB_Profiling.T

In [ ]:
profiling_output

## RFM SEGMENTATION


In [ ]:
# Creating quartiles

quantiles = seg_df[['frequency','monetary','recency' ]].quantile(q=[0.25,0.5,0.75])
quantiles

In [ ]:
quantiles = quantiles.to_dict()
quantiles

In [ ]:
#We create two classes for the RFM segmentation since, being high recency is bad, while high frequency and monetary value is good.
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def RClass(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def FMClass(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [ ]:
seg_df['R_Quartile'] = seg_df['recency'].apply(RClass, args=('recency',quantiles,))
seg_df['F_Quartile'] = seg_df['frequency'].apply(FMClass, args=('frequency',quantiles,))
seg_df['M_Quartile'] = seg_df['monetary'].apply(FMClass, args=('monetary',quantiles,))

In [ ]:
seg_df.head()

In [ ]:
seg_df1=seg_df.assign(R_seg=np.nan, FM_seg =np.nan)
seg_df1.R_seg[seg_df['R_Quartile']==4] = "3_Churn"
seg_df1.R_seg[seg_df['R_Quartile']==2] = "2_At_Risk"
seg_df1.R_seg[seg_df['R_Quartile']==3] = "2_At_Risk"
seg_df1.R_seg[seg_df['R_Quartile']==1] = "1_Active"

seg_df1.FM_seg[(seg_df['F_Quartile']==3) & (seg_df['M_Quartile']==1)] = "2_Gold"
seg_df1.FM_seg[(seg_df['F_Quartile']==4) & (seg_df['M_Quartile']==1)] = "2_Gold"
seg_df1.FM_seg[(seg_df['F_Quartile']==2) & (seg_df['M_Quartile']==2)] = "2_Gold"
seg_df1.FM_seg[(seg_df['F_Quartile']==2) & (seg_df['M_Quartile']==1)] = "2_Gold"
seg_df1.FM_seg[(seg_df['F_Quartile']==3) & (seg_df['M_Quartile']==2)] = "2_Gold"
seg_df1.FM_seg[(seg_df['F_Quartile']==4) & (seg_df['M_Quartile']==2)] = "1_Silver"
seg_df1.FM_seg[(seg_df['F_Quartile']==1) & (seg_df['M_Quartile']==3)] = "2_Gold"
seg_df1.FM_seg[(seg_df['F_Quartile']==2) & (seg_df['M_Quartile']==3)] = "2_Gold"
seg_df1.FM_seg[(seg_df['F_Quartile']==3) & (seg_df['M_Quartile']==3)] = "1_Silver"
seg_df1.FM_seg[(seg_df['F_Quartile']==4) & (seg_df['M_Quartile']==3)] = "1_Silver"
seg_df1.FM_seg[(seg_df['F_Quartile']==1) & (seg_df['M_Quartile']==4)] = "2_Gold"
seg_df1.FM_seg[(seg_df['F_Quartile']==1) & (seg_df['M_Quartile']==2)] = "2_Gold"
seg_df1.FM_seg[(seg_df['F_Quartile']==2) & (seg_df['M_Quartile']==4)] = "1_Silver"
seg_df1.FM_seg[(seg_df['F_Quartile']==3) & (seg_df['M_Quartile']==4)] = "1_Silver"
seg_df1.FM_seg[(seg_df['F_Quartile']==4) & (seg_df['M_Quartile']==4)] = "0_Standard"

seg_df1.FM_seg.fillna("4_Premium", inplace=True)

In [ ]:
seg_df1.head(5)

In [ ]:
# Key performace variable selection
col_kpi=['monetary','recency','frequency']

In [ ]:
#Profiling
Profiling=seg_df1.groupby(['R_seg', 'FM_seg']).apply(lambda x: x[col_kpi].mean()).T

In [ ]:
#Need to retrive Premium customers who are churn now

seg_df1[((seg_df1.R_seg =="3.Churn") & (seg_df1.FM_seg =="4.Premium")) ]

### Scientific (using K-means)


In [ ]:
import scipy.stats as stats
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA  

In [ ]:
seg_df2 = seg_df.loc[:,['recency','frequency','monetary']]
seg_df2.head()

In [ ]:
#standardize the data

sc = StandardScaler()
sc = sc.fit(seg_df2)
seg_df2_std = pd.DataFrame(sc.transform(seg_df2), columns = seg_df2.columns)
seg_df2_std.head(2)

In [ ]:
#KMeans Clustering   input: standardized data, number of cluster
km_3 = KMeans(n_clusters=3, random_state=123).fit(seg_df2)
km_4 = KMeans(n_clusters=4, random_state=123).fit(seg_df2)
km_5 = KMeans(n_clusters=5, random_state=123).fit(seg_df2)
km_6 = KMeans(n_clusters=6, random_state=123).fit(seg_df2)
km_7 = KMeans(n_clusters=7, random_state=123).fit(seg_df2)
km_8 = KMeans(n_clusters=8, random_state=123).fit(seg_df2)

In [ ]:
seg_df2_std['Cluster_3'] = km_3.labels_
seg_df2_std['Cluster_4'] = km_4.labels_
seg_df2_std['Cluster_5'] = km_5.labels_
seg_df2_std['Cluster_6'] = km_6.labels_
seg_df2_std['Cluster_7'] = km_7.labels_
seg_df2_std['Cluster_8'] = km_8.labels_

In [ ]:
print('km_3 =',silhouette_score(seg_df2_std, km_3.labels_))
print('km_4 =',silhouette_score(seg_df2_std, km_4.labels_))
print('km_5 =',silhouette_score(seg_df2_std, km_5.labels_))
print('km_6 =',silhouette_score(seg_df2_std, km_6.labels_))
print('km_7 =',silhouette_score(seg_df2_std, km_7.labels_))
print('km_8 =',silhouette_score(seg_df2_std, km_8.labels_))

In [ ]:
km_3 = 0.289992952858392
km_4 = 0.28820928306705296
km_5 = 0.4619249999257478
km_6 = 0.4638350257297312
km_7 = 0.6547680206859893
km_8 = 0.6513134752259635

In [ ]:
predect_seg = pd.DataFrame(seg_df2_std.Cluster_4 ).rename(columns={'Cluster_4':'Pred_seg'})
predect_seg

In [ ]:
seg_kmeans = pd.concat([seg_df, predect_seg], axis=1)
seg_kmeans.drop(columns = ['decile','R_Quartile','F_Quartile','M_Quartile'], axis=1,inplace = True)

## 4. Predicting customers lifetime value

In [ ]:
# Calculating Average Customer Lifespan in days

cust_month = final_data[['CustomerID','Transaction_Date']]

In [ ]:
cust_month1 = cust_month.groupby('CustomerID')['Transaction_Date'].min().reset_index()
cust_month2 = cust_month.groupby('CustomerID')['Transaction_Date'].max().reset_index()

In [ ]:
cust_month3 = pd.merge(cust_month1, cust_month2, on='CustomerID' ,how= 'left')

In [ ]:
cust_month3['days_diff'] = (cust_month3.Transaction_Date_y- cust_month3.Transaction_Date_x).dt.days

In [ ]:
# Average Customer Lifespan 

ACL = cust_month3.days_diff.mean()/ cust_month3.CustomerID.count()
ACL

In [ ]:
cust_data.head(2)

In [ ]:
seg_df1.head(2)

In [ ]:
cust_rev = seg_df1

In [ ]:
cust_rev.rename(columns={'monetary': 'Revenue'}, inplace=True)

In [ ]:
# CLTV is customer lifetime value
cust_rev['CLTV'] = cust_rev.Revenue * ACL

In [ ]:
def lifetime_value (x):
    if x.CLTV < 50 :
        return 2
    elif x.CLTV < 200:
        return 1
    else :
        return 0
    
# High value = 1
# Medium value = 2
# Low value = 3

In [ ]:
cust_rev['CLTV'] = cust_rev.apply(lifetime_value, axis=1)

In [ ]:
cust_rev

In [ ]:
cust_rev.CLTV.value_counts()

In [ ]:
cust_data_new = pd.merge(cust_data,cust_rev, how='left' ,on='CustomerID')
cust_data_new

In [ ]:
# Creating dummie variables
cust_data_new = pd.get_dummies(cust_data_new, columns=['Location'], drop_first=True, prefix='Location')
cust_data_new = pd.get_dummies(cust_data_new, columns=['Gender'], drop_first=True, prefix='Gender')
cust_data_new = pd.get_dummies(cust_data_new, columns=['R_seg'], drop_first=True, prefix='R_seg')
cust_data_new = pd.get_dummies(cust_data_new, columns=['FM_seg'], drop_first=True, prefix='FM_seg')

In [ ]:
cust_data_new.head(2)

### Balancing the data


In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
x = cust_data_new.loc[:,cust_data_new.columns.difference(['CLTV'])]
y = cust_data_new.CLTV

In [ ]:
#balancing the data
sm = SMOTE()
resampled_X, resampled_y = sm.fit_resample(x, y)

In [ ]:
resampled_y.value_counts()

In [ ]:
cust_data_new2 = pd.concat([resampled_X, resampled_y], axis=1)

In [ ]:
cust_data_new2.head(2)

In [ ]:
cust_data_new2.rename(columns={'Location_New Jersey':'Location_New_Jersey',
                               'Location_New York':'Location_New_York',
                               'Location_Washington DC':'Location_Washington_DC'},inplace=True)

### Feature selection based on importance using F - Regression

In [ ]:
# Feature Selection based on importance
from sklearn.feature_selection import f_regression

In [ ]:
# splitting the data: separate out the feature/input/independant columns and dependant variable
feature_columns = cust_data_new2.columns.difference(['CLTV','CustomerID'])
feature_columns

In [ ]:
# seperate the X and y columns
features = cust_data_new2[feature_columns]
target = cust_data_new2.CLTV

# do the f_regression
F_values, p_values  = f_regression( features, target )

In [ ]:
# combine the output in dataframe
F_regression_op = pd.DataFrame([feature_columns, F_values.round(2), p_values.round(4)]).T

# add the column names
F_regression_op.columns = ['Features', 'F_values', 'p_values' ]
F_regression_op

In [ ]:
# output of the f_regression
feature_columns = list( F_regression_op.loc[ F_regression_op.p_values <= 0.1, 'Features' ] )

In [ ]:
print('No. of feature = ' ,len(feature_columns))
print(feature_columns)

#### VIF (Variance Inflation Factor): Check the multicollinieirity for all the variables in the model


In [ ]:
# import the packages for vif
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices

In [ ]:
model_param = 'CLTV ~ ' + ' + '.join(feature_columns)
model_param

In [ ]:
# separate the Y and X variables
y, X = dmatrices( model_param, cust_data_new2, return_type = 'dataframe' )

# For each X, calculate VIF and save in dataframe
vif = pd.DataFrame()

vif['Features'] = X.columns
vif['VIF Factor'] = [ variance_inflation_factor(X.values, i) for i in range(X.shape[1]) ]

# display the output
vif.round(1)

In [ ]:
#Selecting the feature columns with vif <= 5
feature_columns = list( vif.loc[ vif['VIF Factor'] <= 5, 'Features' ] )
print('No. of feature_columns = ', len(feature_columns))
feature_columns

## Building model


In [ ]:
#importing package
from sklearn.model_selection import train_test_split

#Considering Significant Variables for Machine Learning algorithms
train_x, test_x, train_y, test_y = train_test_split(cust_data_new2[feature_columns],cust_data_new2.CLTV, test_size=0.3, random_state=111)

## Decision Trees


In [ ]:
#Importing packages
from sklearn.tree import DecisionTreeClassifier, export_graphviz, export

In [ ]:
#Model building
param_grid = {'max_depth': np.arange(2, 50),
             'max_features': np.arange(4,100)}
data_tree = GridSearchCV(DecisionTreeClassifier(), param_grid, cv = 3,n_jobs=-1, verbose=True)
data_tree.fit(train_x,train_y)

In [ ]:
print('Best parameters : ',data_tree.best_params_)
print('Accuracy : ' ,data_tree.best_score_)

In [ ]:
#checking accuracy for trian data
print(metrics.classification_report(train_y,data_tree.predict(train_x)))

In [ ]:
#checking accuracy for trian data
print(metrics.classification_report(test_y,data_tree.predict(test_x)))

#### Decision Tree model is giving 77% accuracy for Test Data


## XGBOOST MODEL


In [ ]:
#Importing Required packages
import xgboost

In [ ]:
pargrid_xg = {'n_estimators': [60,70,80,90,100],
               'max_depth':[2,3,4,5,6,7,8,9,10]}

gscv_xg = GridSearchCV(estimator=xgboost.XGBClassifier(), 
                        param_grid=pargrid_xg, 
                        cv=3,n_jobs=-1, verbose=1)

data_xg=gscv_xg.fit(train_x, train_y)

In [ ]:
print('Best score : ' ,gscv_xg.best_params_)
print('Accuracy : ', gscv_xg.best_score_)

In [ ]:
#checking accuracy for trian data
print(metrics.classification_report(train_y,data_tree.predict(train_x)))

In [ ]:
#checking accuracy for trian data
print(metrics.classification_report(test_y,gscv_xg.predict(test_x)))

#### Xgboost Model is giving 75% accuracy on Test data


## 5. Cross-Selling


In [ ]:
#Importing packages
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [ ]:
online_sales.head(2)

In [ ]:
online_sales.Product_Description.value_counts()

In [ ]:
# Clean up spaces in description 
online_sales['Product_Description'] = online_sales['Product_Description'].str.strip()

In [ ]:
#To keeping data set is small, Filtering the data for only one country - France
basket =online_sales.groupby(['CustomerID', 'Product_Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('CustomerID')

In [ ]:
basket.head()

In [ ]:
#we need to consolidate the items into 1 transaction per row with each product 1 hot encoded.
# Convert the units to 1 hot encoded values
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1  

In [ ]:
basket_sets = basket.applymap(encode_units)

In [ ]:
basket_sets.head(2)

In [ ]:
# Build up the frequent items
#Now that the data is structured properly, we can generate frequent item sets that have a support of 
#at least 7% (this number was chosen so that I could get enough useful examples):
frequent_itemsets = apriori(basket_sets, min_support=0.2, use_colnames=True)

In [ ]:
frequent_itemsets.sort_values(by='support', ascending=False)

In [ ]:
rules1 = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)
rules1

In [ ]:
#Generate the rules with their corresponding support, confidence and lift:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules

In [ ]:
rules.sort_values(by='confidence', ascending=False )

In [ ]:
# Filtering the dataframe . In this case, look for a lift of >=1 and high confidence of >=0.9:
rules[ (rules['lift'] >= 1) &
       (rules['confidence'] >= 0.9) ]

## 6. Predicting next purchase day


In [ ]:
cust_month3.head(2)

In [ ]:
cust_next_purchased = cust_month3
cust_next_purchased = cust_next_purchased.rename(columns={'days_diff':'next_purchased'} )
cust_next_purchased.head()

In [ ]:
cust_next_purchased =  cust_next_purchased.groupby('CustomerID')['next_purchased'].mean().reset_index()
cust_next_purchased

In [ ]:
def purchased_day (x):
    if x.next_purchased <= 30 :   # Purchased days btw 0-30 = 0
        return 0                        
    elif x.next_purchased <= 60:  # Purchased days btw 30-60 = 1
        return 1                     
    elif x.next_purchased <= 90:  # Purchased days btw 60-90 = 2
        return 2
    else :                        # Purchased days of 90+ = 3
        return 3

In [ ]:
cust_next_purchased['next_purchased'] = cust_next_purchased.apply(purchased_day, axis=1)

In [ ]:
cust_next_purchased.head()

In [ ]:
# Merging data sets

cust_next_purchased = pd.merge(cust_next_purchased, seg_df1 , on='CustomerID' , how='left')
cust_next_purchased = pd.merge(cust_data, cust_next_purchased, on='CustomerID', how= 'left' )
cust_next_purchased.head()

In [ ]:
# Creating dummie variables

cust_next_purchased = pd.get_dummies(cust_next_purchased, columns=['R_seg'], drop_first=True, prefix='R_seg')
cust_next_purchased = pd.get_dummies(cust_next_purchased, columns=['FM_seg'], drop_first=True, prefix='FM_seg')
cust_next_purchased = pd.get_dummies(cust_next_purchased, columns=['Location'], drop_first=True, prefix='Location')
cust_next_purchased = pd.get_dummies(cust_next_purchased, columns=['Gender'], drop_first=True, prefix='Gender')

In [ ]:
cust_next_purchased.head()

In [ ]:
# Rename the columns to correct format
cust_next_purchased.rename(columns={'Location_New Jersey':'Location_New_Jersey',
                               'Location_New York':'Location_New_York',
                               'Location_Washington DC':'Location_Washington_DC'},inplace=True)


In [ ]:
cust_next_purchased.next_purchased.value_counts()

### Balancing the data


In [ ]:
x = cust_next_purchased.loc[:,cust_next_purchased.columns.difference(['next_purchased'])]
y = cust_next_purchased.next_purchased

In [ ]:
#balancing the data
sm = SMOTE()
resampled_X, resampled_y = sm.fit_resample(x, y)

In [ ]:
cust_next_purchased = pd.concat([resampled_X,resampled_y], axis=1)

In [ ]:
cust_next_purchased.next_purchased.value_counts()

In [ ]:
cust_next_purchased.columns.difference(['next_purchased'])

### Feature selection based on importance using F - Regression


In [ ]:
# splitting the data: separate out the feature/input/independant columns and dependant variable
feature_columns = cust_next_purchased.columns.difference(['next_purchased','CustomerID'])
feature_columns

In [ ]:
# seperate the X and y columns
features = cust_next_purchased[feature_columns]
target = cust_next_purchased.next_purchased

# do the f_regression
F_values, p_values  = f_regression( features, target )

In [ ]:
# combine the output in dataframe
F_regression_op = pd.DataFrame([feature_columns, F_values.round(2), p_values.round(4)]).T

# add the column names
F_regression_op.columns = ['Features', 'F_values', 'p_values' ]
F_regression_op

In [ ]:
# output of the f_regression
feature_columns = list( F_regression_op.loc[ F_regression_op.p_values <= 0.1, 'Features' ] )

In [ ]:
print('No. of feature = ' ,len(feature_columns))
print(feature_columns)

## VIF (Variance Inflation Factor): Check the multicollinieirity for all the variables in the model


In [ ]:
# Feature Selection based on importance
from sklearn.feature_selection import f_regression

In [ ]:
model_param = 'next_purchased ~ ' + ' + '.join(feature_columns)
model_param

In [ ]:
# separate the Y and X variables
y, X = dmatrices( model_param, cust_next_purchased, return_type = 'dataframe' )

# For each X, calculate VIF and save in dataframe
vif = pd.DataFrame()

vif['Features'] = X.columns
vif['VIF Factor'] = [ variance_inflation_factor(X.values, i) for i in range(X.shape[1]) ]

# display the output
vif.round(1)

In [ ]:
features = list( vif.loc[ vif['VIF Factor'] <= 5, 'Features' ] )
features

### Model building


In [ ]:
#Considering Significant Variables for Machine Learning algorithms

train_x, test_x, train_y, test_y = train_test_split(cust_next_purchased[feature_columns],cust_next_purchased.next_purchased, test_size=0.3, random_state=111)

### Decision Trees


In [ ]:
#Model building
param_grid1 = {'max_depth': np.arange(2, 50),
             'max_features': np.arange(4,100)}
data_tree1 = GridSearchCV(DecisionTreeClassifier(), param_grid1, cv = 3,n_jobs=-1, verbose=True)
data_tree1.fit(train_x,train_y)

In [ ]:
print('Best parameters : ',data_tree1.best_params_)
print('Accuracy : ' ,data_tree1.best_score_)

In [ ]:
#checking accuracy for trian data
print(metrics.classification_report(train_y,data_tree1.predict(train_x)))

In [ ]:
#checking accuracy for trian data
print(metrics.classification_report(test_y,data_tree1.predict(test_x)))

#### Decision Tree model is giving 73% accuracy on Test Data


### xgboost


In [ ]:
#Importing packages
from sklearn.tree import DecisionTreeClassifier, export_graphviz, export

In [ ]:
pargrid_xg1 = {'n_estimators': [60,70,80,90,100],
               'max_depth':[2,3,4,5,6,7,8,9,10]}

gscv_xg1 = GridSearchCV(estimator=xgboost.XGBClassifier(), 
                        param_grid=pargrid_xg, 
                        cv=3,n_jobs=-1, verbose=1)

data_xg1=gscv_xg1.fit(train_x, train_y)

In [ ]:
print('Best score : ' ,gscv_xg1.best_params_)
print('Accuracy : ', gscv_xg1.best_score_)

In [ ]:
#checking accuracy for trian data
print(metrics.classification_report(train_y,gscv_xg1.predict(train_x)))

In [ ]:
#checking accuracy for trian data
print(metrics.classification_report(test_y,gscv_xg1.predict(test_x)))

#### XGboost model is giving 82% Accuracy on Test data


## 7. Cohort Analysis


In [ ]:
online_sales.head(2)

In [ ]:
online_sales['total_sales'] = online_sales.Quantity * online_sales.Avg_Price
online_sales['Transaction_Date'] = pd.to_datetime(online_sales['Transaction_Date'] , format = '%Y%m%d')
online_sales['Transaction_month']=online_sales.Transaction_Date.apply(lambda x:x.strftime('%Y-%m'))
online_sales.head(2)

In [ ]:
online_sales.set_index('CustomerID', inplace=True)

online_sales['CohortGroup'] = online_sales.groupby(level=0)['Transaction_Date'].min().apply(lambda x: x.strftime('%Y-%m'))
online_sales.reset_index(inplace=True)
online_sales.head()

In [ ]:
datagrouped = online_sales.groupby(['CohortGroup', 'Transaction_month'])

cohorts = datagrouped.agg({'CustomerID': pd.Series.nunique,
                       'Transaction_ID': pd.Series.nunique,
                        'total_sales': pd.Series.sum})

cohorts.rename(columns={'CustomerID': 'Total_Users',
                        'Transaction_ID': 'Total_orders','total_sales':'Revenue'}, inplace=True)
cohorts.head()

In [ ]:
def cohort_period(online_sales):
    
    online_sales['CohortPeriod'] = np.arange(len(online_sales)) + 1
    return online_sales

cohorts = cohorts.groupby(level=0).apply(cohort_period)
cohorts.head()

In [ ]:
cohorts.reset_index(inplace=True)
cohorts.set_index(['CohortGroup', 'CohortPeriod'], inplace=True)

cohort_group_size = cohorts['Total_Users'].groupby(level=0).first()
cohort_group_size.head()

In [ ]:
cohorts['Total_Users'].unstack(0).head()

In [ ]:
cust_retention = cohorts['Total_Users'].unstack(0).divide(cohort_group_size, axis=1)
cust_retention

In [ ]:
plt.figure(figsize=(15,8))
plt.title('Retention by Monthly Cohorts')
sns.heatmap(data=cust_retention,
            annot = True,
            cmap = "Greens",
            vmin = 0.0,
            vmax = list(cust_retention.max().sort_values(ascending = False))[1]+3,
            fmt = '.1f',
            linewidth = 0.3)

plt.savefig("Retention By Monthly Cohorts")
plt.show()